In [1]:
import pandas as pd
import numpy as np

from collections import Counter
import math
import time

import sklearn
from sklearn.model_selection import train_test_split

from simpletransformers.classification import MultiLabelClassificationModel

#### Checking the GPU

In [ ]:
print("GPU Available: {}".format(torch.cuda.is_available()))
n_gpu = torch.cuda.device_count()
print("Number of GPU Available: {}".format(n_gpu))
print("GPU: {}".format(torch.cuda.get_device_name(0)))

## Import the Train Data

In [2]:
df = pd.read_csv('train_data.csv')
df.head(5)

,id,tasting_notes,briny_0,briny_1,briny_2,briny_3,briny_4,briny_5,chemical_0,chemical_1,...,tart_2,tart_3,tart_4,tart_5,vanilla_0,vanilla_1,vanilla_2,vanilla_3,vanilla_4,vanilla_5
0,5,This rye beast is crammed with so many spices ...,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,6,The aromatics of this brandy have a strong app...,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,7,Sweet spices and floral aromas are found as yo...,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,11,"The nose is floral, with honey, hay, citrus ze...",1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,12,The nose is at first difficult to find behind ...,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [3]:
df['labels'] = tuple(df.iloc[:,2:].values)
df['text'] = df['tasting_notes'].apply(lambda x: x.replace('\n', ' '))

df.sample(5)

,id,tasting_notes,briny_0,briny_1,briny_2,briny_3,briny_4,briny_5,chemical_0,chemical_1,...,tart_4,tart_5,vanilla_0,vanilla_1,vanilla_2,vanilla_3,vanilla_4,vanilla_5,labels,text
3509,8258,"The nose offers a rich sweetness, backed up by...",1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,"(1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","The nose offers a rich sweetness, backed up by..."
3096,7274,This whisky is a distilled crescendo. It start...,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,"(0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",This whisky is a distilled crescendo. It start...
1309,3073,"There is a sweet, smoky style to the aroma wit...",0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,"(0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","There is a sweet, smoky style to the aroma wit..."
1652,3881,"Bright gold in color it has a hot, intense nos...",1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,"(1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","Bright gold in color it has a hot, intense nos..."
2392,5601,Noticeably spicy and dark for a wheated bourbo...,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,"(1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",Noticeably spicy and dark for a wheated bourbo...


In [4]:
label_num = len(df['labels'][0])
#print(label_num)

#### Spliting the Data

In [5]:
train_df, eval_df = train_test_split(df, test_size=0.2)

#### Setting Time for Output Naming

In [6]:
def time_stamping():
    
    date = "".join(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()).split(' ')[0].split('-'))
    timestamp = "".join(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()).split(' ')[1].split(':'))
    
    return date +'_'+ timestamp

time_stamp = time_stamping()

## Multi-label Classification Model

In [7]:
model_type = 'xlnet'

pretrained_model = {'xlnet':'xlnet-base-cased', 
                    'distilbert':'distilbert-base-uncased', 
                    'roberta':'distilroberta-base'}

model = MultiLabelClassificationModel(model_type, pretrained_model[model_type], num_labels=label_num,
                      args={'train_batch_size':8, 'gradient_accumulation_steps':1, 'learning_rate': 3e-5,            
                      'num_train_epochs':2, 'max_seq_length': 256, 'do_lower_case': False, 
                     'overwrite_output_dir': True, 'reprocess_input_data': True, 'evaluate_during_training' : True,
                    'evaluate_during_training_verbose':True, "output_dir": f'outputs/all_labels/{time_stamp}-{model_type}'}, 
                    use_cuda=True) 

## Training

In [8]:
# Train the model.
model.train_model(train_df, eval_df=eval_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 0.691203

C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\optim\lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.509573

...

C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Running loss: 0.195514

C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\optim\lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.222010


C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


## Evaluation

In [9]:
def rounding(num):    
    if num >= 0.5: return 1
    else: return 0
    
def f1_multilabel(labels, preds):
    return sklearn.metrics.f1_score(labels, list(list(map(rounding, i)) for i in preds), average='weighted', zero_division='warn')

def precision_multilabel(labels, preds):
    return sklearn.metrics.precision_score(labels, list(list(map(rounding, i)) for i in preds), average='weighted', zero_division='warn')

def recall_multilabel(labels, preds):
    return sklearn.metrics.recall_score(labels, list(list(map(rounding, i)) for i in preds), average='weighted', zero_division='warn')

def confusion_matrix_multilabel(labels, preds):
    return sklearn.metrics.multilabel_confusion_matrix(labels, list(list(map(rounding, i)) for i in preds))

result, model_outputs, wrong_predictions = model.eval_model(eval_df, recall=recall_multilabel, precision=precision_multilabel, f1=f1_multilabel, c_matrix=confusion_matrix_multilabel)

print(result)

C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\sklearn\metrics\_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


{'LRAP': 0.8255840231223907, 'recall': 0.5452651900020321, 'precision': 0.5662273152966817, 'f1': 0.5424204940617993, 'c_matrix': array([[[102,  86],
        [ 59, 456]],

       [[567,   0],
        [136,   0]],

       [[666,   0],
        [ 37,   0]],

       [[692,   0],
        [ 11,   0]],

       [[700,   0],
        [  3,   0]],

       [[702,   0],
        [  1,   0]],

       [[  0,  15],
        [  0, 688]],

       [[693,   0],
        [ 10,   0]],

       [[701,   0],
        [  2,   0]],

       [[701,   0],
        [  2,   0]],

       [[703,   0],
        [  0,   0]],

       [[702,   0],
        [  1,   0]],

       [[ 80,  29],
        [ 28, 566]],

       [[648,   0],
        [ 55,   0]],

       [[672,   0],
        [ 31,   0]],

       [[684,   0],
        [ 19,   0]],

       [[699,   0],
        [  4,   0]],

       [[703,   0],
        [  0,   0]],

       [[ 40,   5],
        [  1, 657]],

       [[696,   0],
        [  7,   0]],

       [[687,   0],
        [ 

## Prediction / Testing

In [10]:
test_df = pd.read_csv('test_data.csv')
to_predict = test_df.tasting_notes.tolist()

predictions, model_outputs = model.predict(to_predict)
print(len(predictions))

C:\Users\Henry Yao\.conda\envs\transformers\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



1289


## Submission

In [11]:
sub_columns = df.iloc[:,2:-2].columns.tolist()
sub_df = pd.DataFrame(predictions, columns=sub_columns)

sub_df.insert(0, 'id', range(len(test_df)))

In [12]:
sub_df = test_df.merge(sub_df)

In [13]:
sub_df.sample(5)

,id,name,category,tasting_notes,briny_0,briny_1,briny_2,briny_3,briny_4,briny_5,...,tart_2,tart_3,tart_4,tart_5,vanilla_0,vanilla_1,vanilla_2,vanilla_3,vanilla_4,vanilla_5
164,1075,Loch Lomond 12 Year,Whiskey,There's an intriguing balance of fruit and smo...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121,830,Pusser's Rum 15 Year (DISCONTINUED),Rum,The aroma shows dark sweet notes with some rum...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65,434,Clyde May's Special Reserve Whiskey,Whiskey,"Apple cider and apple juice galore, as the inf...",1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
66,435,Familia Camarena Silver Tequila,Tequila,"The nose is full of bright, baked agave notes ...",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,329,Old Forester Birthday Bourbon 2016,Whiskey,The nose is wood-heavy with both oak and cedar...,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [14]:
sub_df.to_csv(f'outputs/all_labels/{time_stamp}-{model_type}/submission.csv', index=False)